## Collect Metadata from Michigan PLSS Plats Survey Maps

This Jupyter Notebook is intended to collect useful metadata from separated `OPEX` files of Michigan PLSS Plats. The contents for these `OPEX` files are formatted as `XML`. Thus, in this documentation, we mainly use the **BeautifulSoup** python library to help pull data out of it.

### file structure
- **harvest.ipynb**
- **RG_87-155_GLO_Survey_Maps_1816-1860** contains different subdirectories for all Michigan townships. There might be one or more than one `.pax.zip.opex` file indicating the plat of each township.
- **bbox of MI townships.csv** offers the bounding box for each Michigan township.
- **metadata.csv** is the output CSV file with all plats' metadata

> Original created on May 27 2021
> @author: Gene/Ziying Cheng @Ziiiiing


In [1]:
import os
from bs4 import BeautifulSoup
import time
import csv
import geocoder

In [2]:
subdirs =  os.listdir('RG_87-155_GLO_Survey_Maps_1816-1860') 

fieldnames = ['Title', 'Alternative Title', 'Description', 'Language', 'Creator', 'Publisher',
              'Resource Type', 'Date Issued', 'Temporal Coverage', 'Date Range',
              'Spatial Coverage', 'Bounding Box', 'Information', 'Download', 'Image', 
              'Identifier', 'ID', 'Access Rights', 'Provider', 'Code', 'Member Of', 'Status', 
              'Accrual Method', 'Date Accessioned', 'Rights', 'Resource Class', 'Format',
              'Suppressed', 'Child Record'] 
              

In [3]:
bboxDict = {}
with open('bbox of MI townships.csv') as fr:
    reader = csv.reader(fr)
    fields = next(reader)
    for row in reader:
        bboxDict[row[11]] = row[12]

In [18]:
def collectMetadata(data):
    metadata = []
    soup = BeautifulSoup(data,'xml')
    
    alternativeTitle = soup.find('title').get_text()
    description = soup.find('abstract').get_text()
    language = soup.find('languageTerm').get_text()
    if language == 'English':
        language = 'eng'

    creator = ''
    creatorElem = soup.find_all(displayLabel="Surveyor")
    for elem in creatorElem:
        if creator:
            creator += '|' + elem.get_text()
        else:
            creator = elem.get_text()

    publisher = soup.find(displayLabel="Agency").get_text()
        
    try:
        dateIssued = soup.find('dateCreated').get_text()
    except:
        dateIssued = ''        
        
    township = soup.find('township').get_text()
    county = soup.find('county').get_text()
    spatialCoverage = '{0}, {1}, Michigan|{1}, Michigan|Michigan'.format(township, county)
        
    try:
        TWNSHPLAB =  subdir.split('_')[0].strip('0') + ' ' + subdir.split('_')[1].strip('0')
        bbox = bboxDict[TWNSHPLAB]
    except:
        bbox = ''
    
    identifier = soup.find('Identifier').get_text()
    ID = soup.find('SourceID').get_text()
    information = 'https://michiganology.org/uncategorized/IO_' + ID
    download = 'https://michiganology.org/download/file/IO_' + ID
    image = 'https://michiganology.org/download/thumbnail/IO_' + ID
    
    rights = soup.find(type="restriction on access").get_text() + ' ' + soup.find(type="use").get_text()
    
    title = ''
    temporalCoverage = ''
    dateRange = ''
    resourceType = 'Cadastral maps'
    accessRights = 'Public'
    provider = 'Michigan State University'
    code = '06a-03'
    memberOf = '06a-03'
    status = 'Active'
    accrualMethod = 'OPEX'
    dateAccessioned = time.strftime('%Y-%m-%d')
    resourceClass = 'Maps'
    format = 'JPEG2000'
    suppressed = 'FALSE'
    childRecord = 'FALSE'
    
    
    metadata = [title, alternativeTitle, description, language, creator, publisher, resourceType,
                dateIssued, temporalCoverage, dateRange,spatialCoverage, bbox, information,
                download, image, identifier, ID, accessRights, provider, code, memberOf, status,
                accrualMethod, dateAccessioned, rights, resourceClass, format, suppressed, childRecord]

    All_Metadata.append(metadata)

In [19]:
All_Metadata = []
count = 0
for subdir in subdirs:
    # skip hidden files
    if subdir.startswith('.') or subdir.endswith('.opex'):
        print(subdir)
        continue
    
    
    print('SEARCHING {}'.format(subdir))
    dirpath = 'RG_87-155_GLO_Survey_Maps_1816-1860/{}'.format(subdir)
    filepaths = ['{}/{}'.format(dirpath, f) for f in os.listdir(dirpath) if f.endswith('.pax.zip.opex')]
    for filepath in filepaths:
        print('>>> reading {}'.format(filepath.split('/')[-1]))
        count += 1
        with open(filepath) as fr:
            contents = fr.read()
            collectMetadata(contents)

SEARCHING 45N_26W_-_Survey_Map_of_Forsyth_Township_Marquette_County
>>> reading 2208.pax.zip.opex
SEARCHING 34N_27W_-_Survey_Map_of_Mellen_Township_Menominee_County
>>> reading 2746.pax.zip.opex
SEARCHING 08N_08E_-_Survey_Map_or_Richfield_Township_Genesee_County
>>> reading 1836.pax.zip.opex
SEARCHING 47N_42W_-_Survey_Map_of_Mareninsco_Township_Gogebic_County
>>> reading 2456.pax.zip.opex
SEARCHING 43N_32W_-_Survey_Map_of_Crystal_Falls_Township_Iron_County
>>> reading 2948.pax.zip.opex
>>> reading 2949.pax.zip.opex
SEARCHING 48N_38W_-_Survey_Map_of_Stannard_Township_and_Interior_Township_
>>> reading 2562.pax.zip.opex
SEARCHING 12N_03W_-_Survey_Map_of_Pine_River_Township_Gratiot_County
>>> reading 719.pax.zip.opex
SEARCHING 34N_02E_-_Survey_Map_of_Allis_Township_Presque_Isle_County
>>> reading 1131.pax.zip.opex
SEARCHING 32N_03W_-_Survey_Map_of_Corwith_Township_Otsego_County
>>> reading 1072.pax.zip.opex
SEARCHING 02N_07W_-_Survey_Map_of_Maple_Grove_Township_Barry_County
>>> reading 34

SEARCHING 48N_16W_-_Survey_Map_of_Burt_Township_Alger_County
>>> reading 2172.pax.zip.opex
SEARCHING 15N_09W_-_Survey_Map_of_Colfax_Township_Mecosta_County
>>> reading 615.pax.zip.opex
>>> reading 616.pax.zip.opex
SEARCHING 23N_06E_-_Survey_Map_of_Plainfield_Township_Iosco_County
>>> reading 1257.pax.zip.opex
>>> reading 1256.pax.zip.opex
>>> reading 1255.pax.zip.opex
SEARCHING 30N_08W_-_Survey_Map_of_Forest_Home_Township_Antrim_County
>>> reading 845.pax.zip.opex
SEARCHING 04S_06W_-_Survey_Map_of_Tekonsha_Township_Calhoun_County
>>> reading 188.pax.zip.opex
>>> reading 187.pax.zip.opex
SEARCHING 35N_01E_-_Survey_Map_of_Waverly_Township_Cheboygan_County
>>> reading 1108.pax.zip.opex
SEARCHING 47N_43W_-_Survey_Map_of_Mareninsco_Township_Gogebic_County
>>> reading 2448.pax.zip.opex
SEARCHING 43N_33W_-_Survey_Map_of_Crystal_Falls_Township_Iron_County
>>> reading 2952.pax.zip.opex
>>> reading 2951.pax.zip.opex
SEARCHING 08S_08W_-_Survey_Map_of_Nobel_Township_Branch_County
>>> reading 234.p

SEARCHING 27N_09W_-_Survey_Map_of_the_Southern_half_of_Whitewater_Township
>>> reading 897.pax.zip.opex
>>> reading 896.pax.zip.opex
SEARCHING 08S_05E_-_Survey_Map_of_Riga_Township_Lenawee_County
>>> reading 1795.pax.zip.opex
SEARCHING 11N_15E_-_Survey_Map_of_Washington_Township_Sanilac_County
>>> reading 1526.pax.zip.opex
SEARCHING 12N_06W_-_Survey_Map_of_Home_Township_Montcalm_County
>>> reading 663.pax.zip.opex
SEARCHING 18N_11W_-_Survey_Map_of_Pinora_Township_Lake_County
>>> reading 582.pax.zip.opex
SEARCHING 07S_04W_-_Survey_Map_of_Reading_Township_Hillsdale_County
>>> reading 223.pax.zip.opex
SEARCHING 51N_29W_-_Survey_Map_of_Michigamme_Township_and_Powell_Township_
>>> reading 2273.pax.zip.opex
SEARCHING 45N_18W_-_Survey_Map_of_Hiawatha_Township_Schoolcraft_County
>>> reading 2198.pax.zip.opex
SEARCHING 09N_14E_-_Survey_Map_of_Speaker_Township_Sanilac_County
>>> reading 1552.pax.zip.opex
SEARCHING 28N_03E_-_Survey_Map_of_Clinton_Township_Oscoda_County
>>> reading 1172.pax.zip.op

RG_87-155_GLO_Survey_Maps_1816-1860.opex
SEARCHING 49N_11W_-_Survey_Map_of_Mc_Millan_Township_Luce_County
>>> reading 2121.pax.zip.opex
SEARCHING 22N_09E_-_Survey_Map_of_Baldwin_Township_Iosco_County
>>> reading 1823.pax.zip.opex
SEARCHING 29N_12W_-_Survey_Map_of_Centerville_Township_Leelanau_County
>>> reading 895.pax.zip.opex
SEARCHING 27N_09E_-_Survey_Map_of_Haynes_Township_Alcona_County
>>> reading 1802.pax.zip.opex
SEARCHING 46N_25W_-_Survey_Map_of_Sands_Township_Marquette_County
>>> reading 2296.pax.zip.opex
SEARCHING 27N_04E_-_Survey_Map_of_Comins_Township_Oscoda_County
>>> reading 1174.pax.zip.opex
SEARCHING 02N_10E_-_Survey_Map_of_Bloomfield_Township_Oakland_County
>>> reading 1839.pax.zip.opex
SEARCHING 49N_31W_-_Survey_Map_of_Spurr_Township_Baraga_County
>>> reading 2967.pax.zip.opex
>>> reading 2966.pax.zip.opex
SEARCHING 47N_16W_-_Survey_Map_of_Hiawatha_Township_Schoolcraft_County
>>> reading 2083.pax.zip.opex
>>> reading 2084.pax.zip.opex
>>> reading 2082.pax.zip.opex
>>>

>>> reading 1637.pax.zip.opex
>>> reading 1636.pax.zip.opex
SEARCHING 33N_09W_-_Survey_Map_of_Norwood_Township_Charlevoix_County
>>> reading 1020.pax.zip.opex
>>> reading 1021.pax.zip.opex
SEARCHING 08N_03E_-_Survey_Map_of_New_Haven_Township_Shiawassee_County
>>> reading 1559.pax.zip.opex
SEARCHING 11N_08E_-_Survey_Map_of_Vassar_Township_Tuscola_County
>>> reading 1466.pax.zip.opex
SEARCHING 31N_11W_-_Survey_Map_of_Leelanau_Township_Leelanau_County
>>> reading 2834.pax.zip.opex
>>> reading 2833.pax.zip.opex
SEARCHING 10N_10E_-_Survey_Map_of_Rich_Township_Lapeer_County
>>> reading 1827.pax.zip.opex
SEARCHING 13N_16W_-_Survey_Map_of_Otto_Township_Oceana_County
>>> reading 556.pax.zip.opex
SEARCHING 27N_08W_-_Survey_Map_of_Kalkaska_Township_Kalkaska_County
>>> reading 859.pax.zip.opex
>>> reading 862.pax.zip.opex
>>> reading 860.pax.zip.opex
>>> reading 861.pax.zip.opex
SEARCHING 50N_33W_-_Survey_Map_of_LAnse_Township_Baraga_County
>>> reading 2884.pax.zip.opex
>>> reading 2604.pax.zip.op

...

>>> reading 726.pax.zip.opex
SEARCHING 26N_03W_-_Survey_Map_of_Grayling_Township_Crawford_County
>>> reading 756.pax.zip.opex
SEARCHING 44N_15W_-_Survey_Map_of_Manistique_Township_Schoolcraft_County
>>> reading 2177.pax.zip.opex
SEARCHING 06S_13W_-_Survey_Map_of_Newberg_Township_Cass_County
>>> reading 96.pax.zip.opex
SEARCHING 52N_34W_-_Survey_Map_of_Portage_Township_Houghton_County_and_Bar
>>> reading 2479.pax.zip.opex
SEARCHING 26N_06E_-_Survey_Map_of_Millen_Township_Alcona_County
>>> reading 1311.pax.zip.opex
SEARCHING 08S_04E_-_Survey_Map_of_Ogden_Township_Lenawee_County
>>> reading 1794.pax.zip.opex
SEARCHING 04N_15E_-_Survey_Map_of_Casco_Township_St._Clair_County
>>> reading 1400.pax.zip.opex
>>> reading 1401.pax.zip.opex
SEARCHING 03S_01E_-_Survey_Map_of_Napoleon_Township_and_Leoni_Township_Jac
>>> reading 1622.pax.zip.opex
SEARCHING 49N_36W_-_Survey_Map_of_Laird_Township_Houghton_County
>>> reading 2900.pax.zip.opex
SEARCHING 16N_05W_-_Survey_Map_of_Gilmore_Township_Isabella_C

>>> reading 1990.pax.zip.opex
SEARCHING 17N_07W_-_Survey_Map_of_Orient_Township_Osceola_County
>>> reading 645.pax.zip.opex
SEARCHING 10N_02W_-_Survey_Map_of_North_Star_Township_Gratiot_County
>>> reading 242.pax.zip.opex
SEARCHING 08S_06W_-_Survey_Map_of_Kinderhook_Township_Branch_County
>>> reading 232.pax.zip.opex
SEARCHING 30N_02E_-_Survey_Map_of_Briley_Township_Montmorency_County
>>> reading 1178.pax.zip.opex
SEARCHING 17N_04E_-_Survey_Map_of_Pinconning_Township_Bay_County
>>> reading 1337.pax.zip.opex
SEARCHING 31N_01W_-_Survey_Map_of_Charlton_Township_Otsego_County
>>> reading 1082.pax.zip.opex
SEARCHING 29N_07W_-_Survey_Map_of_Custer_Township_Antrim_County
>>> reading 839.pax.zip.opex
SEARCHING 47N_03W_-_Survey_Map_of_Bay_Mills_Township_Chippewa_County
>>> reading 2013.pax.zip.opex
>>> reading 2014.pax.zip.opex
>>> reading 2015.pax.zip.opex
>>> reading 2012.pax.zip.opex
SEARCHING 32N_09E_-_Survey_Map_of_Alpena_Township_Alpena_County
>>> reading 1060.pax.zip.opex
>>> reading 105

SEARCHING 40N_21W_-_Survey_Map_of_Ensign_Township_Delta_County
>>> reading 2681.pax.zip.opex
SEARCHING 15N_05W_-_Survey_Map_of_Nottawa_Township_Isabella_County
>>> reading 653.pax.zip.opex
>>> reading 652.pax.zip.opex
SEARCHING 39N_03W_-_Survey_Map_of_Mackinaw_Township_Cheboygan_County
>>> reading 1094.pax.zip.opex
SEARCHING 16N_03E_-_Survey_Map_of_Garfield_Township_Bay_County
>>> reading 1338.pax.zip.opex
SEARCHING 40N_22W_-_Survey_Map_of_Escanaba_Township_and_Ensign_Township_De
>>> reading 2680.pax.zip.opex
SEARCHING 08N_01E_-_Survey_Map_of_Fairfield_Township_Shiawassee_County
>>> reading 1602.pax.zip.opex
SEARCHING 58N_26W_-_Survey_Map_of_Manitou_Island_Grant_Township_Keweenaw_C
>>> reading 2376.pax.zip.opex
SEARCHING 36N_24W_-_Survey_Map_of_Cedarville_Township_Menominee_County
>>> reading 2736.pax.zip.opex
SEARCHING 08N_13E_-_Survey_Map_of_Lynn_Township_St._Clair_County
>>> reading 1531.pax.zip.opex
SEARCHING 27N_07E_-_Survey_Map_of_Hawes_Township_Alcona_County
>>> reading 1308.pax

>>> reading 343.pax.zip.opex
SEARCHING 44N_19W_-_Survey_Map_of_Au_Train_and_Munising_Townships_Alger_Co
>>> reading 2096.pax.zip.opex
>>> reading 2097.pax.zip.opex
SEARCHING 11N_09W_-_Survey_Map_of_Maple_Valley_Township_Montcalm_County
>>> reading 611.pax.zip.opex
SEARCHING 03S_15W_-_Survey_Map_of_Lawrence_Township_Van_Buren_County
>>> reading 11.pax.zip.opex
SEARCHING 04N_05E_-_Survey_Map_of_Deerfield_Township_Livingston_County
>>> reading 1865.pax.zip.opex
SEARCHING 14N_04W_-_Survey_Map_of_Union_Township_Isabella_County
>>> reading 711.pax.zip.opex
SEARCHING 55N_31W_-_Survey_Map_of_Torch_Lake_Township_Houghton_County
>>> reading 2505.pax.zip.opex
SEARCHING 07S_06E_-_Survey_Map_of_Summerfield_Township_Monroe_County
>>> reading 1800.pax.zip.opex
SEARCHING 03S_17W_-_Survey_Map_of_Coloma_Township_Watervliet_Township_Berr
>>> reading 9.pax.zip.opex
SEARCHING 46N_22W_-_Survey_Map_of_Rock_River_Township_and_Onota_Township_A
>>> reading 2267.pax.zip.opex
SEARCHING 09S_01W_-_Survey_Map_of_Wri

SEARCHING 35N_05E_-_Survey_Map_of_Rogers_Township_Presque_Isle_County
>>> reading 1114.pax.zip.opex
SEARCHING 38N_19W_-_Survey_Map_of_Fairbanks_Township_Delta_County
>>> reading 2704.pax.zip.opex
SEARCHING 10N_02E_-_Survey_Map_of_Brant_Township_Saginaw_County
>>> reading 1580.pax.zip.opex
SEARCHING 52N_35W_-_Survey_Map_of_Portage_Township_Houghton_County
>>> reading 2488.pax.zip.opex
SEARCHING 03S_02W_-_Survey_Map_of_Spring_Arbor_Township_Jackson_County
>>> reading 167.pax.zip.opex
SEARCHING 48N_39W_-_Survey_Map_of_Stannard_Township_Ontonagon_County
>>> reading 2561.pax.zip.opex
SEARCHING 41N_17W_-_Survey_Map_of_Inwood_Township_Schoolcraft_County
>>> reading 2239.pax.zip.opex
SEARCHING 47N_26W_-_Survey_Map_of_Richmond_Township_and_Neguanee_Township_
>>> reading 2290.pax.zip.opex
SEARCHING 05S_03W_-_Survey_Map_of_Scipio_Township_Fayette_Township_Hillsda
>>> reading 192.pax.zip.opex
SEARCHING 42N_02W_-_Survey_Map_of_St._Ignace_Township_Mackinac_County
>>> reading 2038.pax.zip.opex
SEARCH

In [22]:
with open('metadata.csv', 'w') as f:
    write = csv.writer(f)
      
    write.writerow(fieldnames)
    write.writerows(All_Metadata)